In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import optuna
import yfinance as yf
import datetime as dt
from time import sleep
from fbprophet import Prophet
from fbprophet.diagnostics import cross_validation, performance_metrics


In [2]:
class TerryD:
    def __init__(self, ticker):
        self.ticker = ticker
        self.today = dt.datetime.today().strftime('%Y%m_%d')
        self.data = self.get_data()
        self.model = self.prophet_model()
        self.df = None
        self.best_params = {}
        self.best_result = {}

    def get_data(self, period='2y', interval='1d'):
        data = yf.download(self.ticker, period=period, interval=interval)
        data = data.reset_index()
        data = data.rename(columns={'Date': 'ds', 'Adj Close': 'y'})
        data = data[['ds', 'y']]
        df = pd.DataFrame(data)
        return df

    def prophet_model(self,):
        model = Prophet(growth='linear')
        params = self.best_params
        if params:
            model.seasonality_prior_scale = params['seasonality_prior_scale']
            model.holidays_prior_scale = params['holidays_prior_scale']
            model.changepoint_prior_scale = params['changepoint_prior_scale']
        model.fit(self.data)
        future = model.make_future_dataframe(periods=90)
        forecast = model.predict(future)
        return forecast

    
    def objective(self, trial):
        params = {
            'seasonality_prior_scale': trial.suggest_loguniform('seasonality_prior_scale', 0.01, 10.0),
            'holidays_prior_scale': trial.suggest_loguniform('holidays_prior_scale', 0.01, 10.0),
            'changepoint_prior_scale': trial.suggest_loguniform('changepoint_prior_scale', 0.001, 0.2)
        }
        forecast = self.prophet_model(self.data, params)
        metrics = performance_metrics(forecast)
        rmse = metrics['rmse']
        return rmse

    def optimize(self, n_trials=100):
        study = optuna.create_study()
        study.optimize(self.objective, n_trials=n_trials)
        self.best_params = study.best_params
        self.best_result = study.best_value

    def plot(self):
        self.model.plot(self.model)


In [3]:
forecaster = TerryD("AAPL")
forecaster.optimize()
print("Best hyperparameters:", forecaster.best_params)
print("Best result (RMSE):", forecaster.best_result)


[*********************100%***********************]  1 of 1 completed


AttributeError: 'TerryD' object has no attribute 'best_params'